## PyTorch implementation of Deep Bilteral Learning for Real Time Image Enhancement

In [2]:
from __future__ import division, print_function, unicode_literals
import numpy as np
from PIL import Image
import cv2
import os, sys, glob
import matplotlib.pyplot as plt
#Torch Imports
import torch
import torchvision
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data
import torchvision.models as models
import torchvision.transforms as transforms

from tensorboardX import SummaryWriter

In [3]:
size = (256, 256)
batch_size = 100
learning_rate = 0.01
num_epochs = 1
img_w = 4000
img_h = 3000

writer = SummaryWriter()
sample_rate = 44100
freqs = [262, 294, 330, 349, 392, 440, 440, 440, 440, 440, 440]

In [15]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, train, transform=None):
        #Init Function
        super(Dataset, self).__init__()
        self.root_dir = root_dir
        self.train = train
        self.transform = transform
        
        self.full_res = []
        self.low_res = []
        self.truth = []
        
        if (train):
            dir = self.root_dir + '/train/'
        else :
            dir = self.root_dir + '/test/'
        
#         for img_path in glob.glob (dir + '*.jpg'):
        for img_path in glob.glob (dir +'input/' '*.tif'):
            img_name = img_path.split('/')[-1]
            print (img_name)
#             himage = Image.open (img_path)
#             himage = himage.resize (img_w,img_h)           
#             limage = himage.resize (size)           
#             output = Image.open(dir+'output/'+img_name)
#             output = output.resize(img_w,img_h)

            himage = cv2.imread (img_path)
            print(himage.shape)
            himage = cv2.resize (himage,(img_w,img_h))
            print(himage.shape)
            limage = cv2.resize (himage,size)
            output = cv2.imread (dir+'output/'+img_name)
            output = cv2.resize (output,(img_w,img_h))

            
            self.full_res.append (himage)
            self.low_res.append (limage)
            self.truth.append (output)

    def __len__(self):
        #Length function ?
        return len(self.full_res)

    def __getitem__(self, idx):
        #Accessor Function
        if self.transform is None:
            return (self.full_res[idx],self.low_res[idx])
        else:
            limg_transformed = self.transform(self.low_res[idx])
            himg_transformed =  self.transform(self.full_res[idx])
            truth_transformed = self.transform(self.truth[idx])
            return (himg_transformed, limg_transformed, truth_transformed)
       

In [17]:
composed_transform = transforms.Compose([transforms.ToTensor()])
# train_dataset = Dataset (root_dir = '../data', train = True, transform = composed_transform)
train_dataset = Dataset (root_dir = '../dataset', train = True, transform = composed_transform)
im, im2, im3 = train_dataset.__getitem__(0)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

a0004-jmac_MG_1384.tif
(2912, 4368, 3)
(3000, 4000, 3)
a0020-jmac_MG_6225.tif
(2912, 4368, 3)
(3000, 4000, 3)
a0006-IMG_2787.tif
(2848, 4272, 3)
(3000, 4000, 3)
a0045-20050320_081337__E6B1177.tif
(2336, 3504, 3)
(3000, 4000, 3)
a0010-jmac_MG_4807.tif
(2336, 3504, 3)
(3000, 4000, 3)
a0041-IMG_4972.tif
(2848, 4272, 3)
(3000, 4000, 3)
a0034-LSYD4O2202.tif
(3328, 4992, 3)
(3000, 4000, 3)
a0001-jmac_DSC1459.tif
(2000, 3008, 3)
(3000, 4000, 3)
a0018-kme_234.tif
(2336, 3504, 3)
(3000, 4000, 3)
a0030-_MG_7844.tif
(2912, 4368, 3)
(3000, 4000, 3)
a0017-050710_031618__MG_3496.tif
(2336, 3504, 3)
(3000, 4000, 3)
a0050-_DSC0006.tif
(2000, 3008, 3)
(3000, 4000, 3)
a0021-07-11-28-at-09h22m57s-_MG_7427.tif
(2912, 4368, 3)
(3000, 4000, 3)


In [102]:
class LocalFeatureNet (nn.Module):
           
    def __init__(self):
        super (LocalFeatureNet, self).__init__()
        
        self.relu  = nn.ReLU (inplace = True)
        
        self.conv1 = nn.Conv2d (in_channels = 3,   out_channels = 8, kernel_size = 3, stride = 2, padding = 1)
        self.conv2 = nn.Conv2d (in_channels = 8,  out_channels = 16,  kernel_size = 3, stride = 2, padding = 1)
        self.conv3 = nn.Conv2d (in_channels = 16, out_channels = 32,  kernel_size= 3, stride = 2,padding = 1)
        self.conv4 = nn.Conv2d (in_channels = 32, out_channels = 64,  kernel_size = 3, stride = 2, padding = 1)
        
        
        self.localconv1 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 1, padding = 1)
        self.localconv2 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size= 3, stride = 1, padding = 1)
        
        self.globalconv1 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 2,padding = 1)
        self.globalconv2 = nn.Conv2d (in_channels = 64, out_channels = 64,  kernel_size = 3, stride = 2, padding = 1)
        
        self.globalfc1 = nn.Linear (1024, 256)
        self.globalfc2 = nn.Linear (256, 128)
        self.globalfc3 = nn.Linear (128, 64)
        self.linear = nn.Conv2d(in_channels = 64, out_channels = 96,  kernel_size = 1, stride = 1)
        
        # Pixel Wise Network
        self.pixelwise_bias = nn.Parameter (torch.rand(3, 1), requires_grad=True)
        self.pixelwise_weight = nn.Parameter (torch.eye(3), requires_grad = True)
        self.pixelwise_obias = nn.Parameter (torch.eye(1), requires_grad = True)
        
        self.relu_slopes = nn.Parameter(torch.rand(3,16), requires_grad = True)
        self.relu_shifts = nn.Parameter(torch.rand(16,3), requires_grad = True)
        
    def custom_relu(self,channel,value):
        print("value",value.size())
        size = (16,value.size()[0],value.size()[1],value.size()[2])
        size_alt = (1L,value.size()[0],value.size()[1],value.size()[2])
        print("size: ",size,"size_alt",size_alt)
        value = value.expand(size)
        
#         print(self.relu_shifts[:,channel])
        a = self.relu_shifts[:,channel].clone()
        a = a.view(16,1,1,1)
#         print(a.data,a)
#         value = self.relu(value - a.repeat(size_alt))    
# Upper line is not working for some damn reason so I hard coded it for now.
#         print ("here ", a.repeat(1,1,img_h,img_w).size())
        value = self.relu(value - a.repeat(size[1],1,img_h,img_w))
        print("st ", value.size())
        print ("st2 ", self.relu_slopes.size())
        value = self.relu_slopes.matmul(value.view(img_h,img_w,16,-1))
        print ("stt ", value.size())
        value = value.view(-1,3,img_h,img_w)
        value = value[:,1,:,:]+value[:,2,:,:]+value[:,0,:,:]
        print ("stt2 ", value.size())
        return value
    
    def upsample(self, grid, bilat):
        sx = sy = 16
        d = 8
#         writer.add_scalar('upsample/sx', sx)
        writer.add_scalars('upsample/sizes', {"sx": sx,
                                              "sy": sy,
                                              "d": d })
#         g = grid.clone() 
        
        return torch.rand(1,img_h,img_w,3,4)
        
    def output(self, grid, inp):
        out = torch.rand(inp.size()[0],img_h,img_w,3)
        out = 0 * out
        grid = grid.view(-1,img_h,img_w,12)
        
        for i in range(0,3):
            out[:,:,:,i] = out[:,:,:,i] + grid[:,:,:,3+4*i]
            for j in range(0,3):
#                 print(grid[:,:,:,j+4*i].size())
#                 print(inp[:,j,:,:].size())
                a = torch.autograd.Variable(grid[:,:,:,j+4*i], requires_grad=False)
#                 a = a.numpy()
                b = inp[:,j,:,:]
#                 b = b.numpy()
                temp = a.data * b.data
#                 print(temp.size())
                out[:,:,:,i] = out[:,:,:,i] + temp
        
        return out
    
    def forward (self, h, l):
        
#         print (x.size())
        
        x = self.relu (self.conv1 (l))
        print ("conv1",x.size())
        x = self.relu (self.conv2 (x))
        print ("conv2",x.size())
        x = self.relu (self.conv3 (x))
        print ("conv3",x.size())
        x = self.relu (self.conv4 (x))
        print ("conv4",x.size())
#         print ("lel")
        y = self.localconv1 (x)
#         print (y.size())
        y = self.localconv2 (y)
        
#         print (y.size())
        
        z = self.globalconv1 (x)
        z = self.globalconv2 (z)
        z = self.globalfc1 (z.view(z.size()[0], -1))
        z = self.globalfc2 (z)
        z = self.globalfc3 (z)
        
#         print (z.size())
        
        
        fused = self.relu(z.view(-1,64,1,1)+y)
#         print (fused.size())
        
        lin = self.linear(fused)
#         print (lin.size())
        
        bilat = lin.view(-1, 8, 3, 4, 16, 16)
        
        
#         bilat = self.bilateralGrid (fused)
        
        print ("bilat",bilat.size())
        
#         return bilat

# pixel wise network
        for i in range(0,3):
            a = self.pixelwise_weight[i,:].view(-1,3)
            b = h.unsqueeze(0).view(3,-1)
            print(a.size(),b.size())
            p = torch.mm(a,b)
            print(p.size())
            p = p.view(h.size()[0],h.size()[2],h.size()[3]) + self.pixelwise_bias[i]
            print(p.size())
#             p = torch.bmm(self.pixelwise_weight[i,:], h.unsqueeze(0).view(3,-1)).view(h.size()) + self.pixelwise_bias[i]
            print(i)
            p += self.custom_relu(i,p)
        p += self.pixelwise_obias
        p = p.view(p.size()[0],1,p.size()[1],p.size()[2])
        print("p",p.size())
        grid = p
        writer.add_image('gridmap',grid)
        bilat_new = self.upsample(grid, bilat)
        
        return self.output(bilat_new,h)
        
        

In [103]:
model = LocalFeatureNet() 

# Add code for using CUDA here if it is available
use_gpu = False
if(torch.cuda.is_available()):
    use_gpu = True
    model.cuda()

# Loss function and optimizers
criterion = torch.nn.MSELoss()# Define MSE loss
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)# Use Adam optimizer, use learning_rate hyper parameter

In [104]:
def train():
    # Code for training the model
    # Make sure to output a matplotlib graph of training losses
    loss_arr = []
    for epoch in range(num_epochs):
        for i, (himage, limage, truth) in enumerate(train_loader):  
            # Convert torch tensor to Variable
            himage = Variable(himage)
            limage = Variable(limage)
            truth = Variable(truth)
            if(use_gpu):
                himage=himage.cuda()
                limage=limage.cuda()
                truth = truth.cuda()
            # Forward + Backward + Optimize
            optimizer.zero_grad()  # zero the gradient buffer
            outputs = model(himage, limage)
            writer.add_image('himage', himage)
            writer.add_image('limage', limage)
            writer.add_image('truth',truth)
            writer.add_image('output', outputs.view(-1,3,img_h,img_w))
#             loss = criterion(outputs, truth)
#             loss.backward()
#             optimizer.step()
#             loss_arr.append(loss.data[0])
#             if (i+1) % batch_size == 0:       
#                 print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
#                        %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
    
#     plt.plot( np.array(range(1,len(loss_arr)+1)), np.array(loss_arr))
#     plt.show()


In [ ]:
%time train()

In [ ]:
# writer.export_scalars_to_json("./all_scalars.json")
writer.close()